Imports

In [1]:
#imports
from tqdm import tqdm

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, pipeline, Trainer, TrainingArguments

from bert_training import make_training_pairs, gold_lookup

from utils import load_data, prepare_all_samples, get_entity_date_pairs, calculate_metrics
from naive_extractor import naive_extraction
from bert_extractor import bert_extraction
from llama_extractor import llama_extraction, make_binary_prompt, parse_llama_answer

Data Loading

In [2]:
# Load data
df = load_data("data/synthetic.csv")
print(f"Loaded {len(df)} records")
#df

Loaded 101 records


In [3]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[0]

Prepared 101 samples


Naive Extractor

In [4]:
#Process samples
predictions = []

for sample in samples:
    relationships = naive_extraction(sample['note_text'], sample['entities_list'], sample['dates'])
    predictions.extend(relationships)

In [5]:
# Look at predictions
predictions

[{'entity_label': 'pituitary_adenoma',
  'entity_category': 'disorder',
  'date': '2024-08-02',
  'confidence': 1.0,
  'distance': 99},
 {'entity_label': 'rheumatoid_arthritis',
  'entity_category': 'disorder',
  'date': '2024-10-23',
  'confidence': 1.0,
  'distance': 96},
 {'entity_label': 'meningitis',
  'entity_category': 'disorder',
  'date': '2024-10-23',
  'confidence': 1.0,
  'distance': 180},
 {'entity_label': 'bronchitis',
  'entity_category': 'disorder',
  'date': '2025-01-29',
  'confidence': 1.0,
  'distance': 87},
 {'entity_label': 'osteoarthritis',
  'entity_category': 'disorder',
  'date': '2004-10-24',
  'confidence': 1.0,
  'distance': 29},
 {'entity_label': 'bronchitis',
  'entity_category': 'disorder',
  'date': '2004-10-24',
  'confidence': 1.0,
  'distance': 144},
 {'entity_label': 'depression',
  'entity_category': 'disorder',
  'date': '2025-05-07',
  'confidence': 1.0,
  'distance': 82},
 {'entity_label': 'schizophrenia',
  'entity_category': 'disorder',
  'dat

In [6]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

{'precision': 0.1277533039647577,
 'recall': 0.13551401869158877,
 'f1': 0.13151927437641722,
 'tp': 29,
 'fp': 198,
 'fn': 185}

BERT Base

In [7]:
# Load local model and tokenizer
#model_path = "./bert_model_training/base_model"
model_path = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Process samples
predictions = []

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        text = f"Does '{pair['entity_label']}' relate to '{pair['date']}'? {sample['note_text'][:200]}..."
        pred, conf = bert_extraction(text, model, tokenizer)
        if pred == 1:
            predictions.append({'entity_label': pair['entity_label'], 'date': pair['date'], 'confidence': conf})

Samples: 100%|██████████| 101/101 [00:05<00:00, 18.33it/s]


In [9]:
#Look at predictions
predictions

[{'entity_label': 'tension_headache',
  'date': '2024-11-20',
  'confidence': 0.504046618938446},
 {'entity_label': 'tension_headache',
  'date': '2020-12-24',
  'confidence': 0.506692111492157},
 {'entity_label': 'bronchitis',
  'date': '2024-11-20',
  'confidence': 0.5081084370613098},
 {'entity_label': 'bronchitis',
  'date': '2020-12-24',
  'confidence': 0.510748565196991},
 {'entity_label': 'fibromyalgia',
  'date': '2024-11-20',
  'confidence': 0.5055006742477417},
 {'entity_label': 'fibromyalgia',
  'date': '2020-12-24',
  'confidence': 0.5090793371200562},
 {'entity_label': 'pituitary_adenoma',
  'date': '2024-11-20',
  'confidence': 0.5024188756942749},
 {'entity_label': 'pituitary_adenoma',
  'date': '2020-12-24',
  'confidence': 0.5070133805274963},
 {'entity_label': 'congenital malformation',
  'date': '2024-11-20',
  'confidence': 0.5078064203262329},
 {'entity_label': 'congenital malformation',
  'date': '2020-12-24',
  'confidence': 0.5107066035270691},
 {'entity_label':

In [10]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

{'precision': 0.14285714285714285,
 'recall': 0.009345794392523364,
 'f1': 0.017543859649122806,
 'tp': 2,
 'fp': 12,
 'fn': 212}

BERT Finetuning

In [ ]:
# Prepare data
df_train = make_training_pairs(samples, gold_lookup)
dataset = Dataset.from_pandas(df_train)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert_model_training/bert_finetuned",
    per_device_train_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,  # For demo; use a real split for real training!
)

In [ ]:
trainer.train()

In [ ]:
predictions = []

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        text = f"Does '{pair['entity_label']}' relate to '{pair['date']}'? {sample['note_text'][:200]}..."
        pred, conf = bert_extraction(text, model, tokenizer)
        if pred == 1:
            predictions.append({'entity_label': pair['entity_label'], 'date': pair['date'], 'confidence': conf})

In [ ]:
predictions

In [ ]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

Llama

In [ ]:
# Load in notebook (you control this)
#generator = pipeline("text-generation", model="../Llama-3.2-3B-Instruct", device=-1)
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device=-1
)

In [ ]:
# Create prompt in notebook
#prompt = f"In the following text, confirm if {text} Answer YES or NO:"
#response = llama_extraction(prompt, generator)

In [ ]:
prompt = "Does the following text indicate a relationship between 'asthma' and '2024-08-02'? Answer YES or NO. Text: Patient diagnosed with asthma on 2024-08-02."
result = generator(prompt)
print(result[0]['generated_text'])

In [ ]:
for sample in tqdm(samples[:1], desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs[:1]:
        #print(pair)
        prompt = make_binary_prompt(pair['entity'], pair['date_info'], sample['note_text'])
        #print(prompt)
        response = llama_extraction(prompt, generator)
        print(response)
        pred, conf = parse_llama_answer(response)
        print(pred, conf)

In [ ]:
predictions = []

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        prompt = make_binary_prompt(pair['entity'], pair['date_info'], sample['note_text'])
        response = llama_extraction(prompt, generator)
        pred, conf = parse_llama_answer(response)
        if pred == 1:
            predictions.append({
                'entity_label': pair['entity_label'],
                'date': pair['date'],
                'confidence': conf
            })

In [ ]:
predictions

In [ ]:
metrics = calculate_metrics(predictions, df)
metrics